This project is to find each preoperty information.
1. if the closest station has direct line to Melbourne Central during weekday from 7 a.m. to 9 a.m. or not. 
2. How long would it take (average).
3. number of houses in each suburb.
4. number of units in each suburb (condo/apartment)
5. municipality of the property suburb(LGA).
6. Percentage of the Australian-born population in the property suburb.
7. median income of the population in the property suburb 
8. population in the property suburb 


In [1]:
import pandas as pd

# Extract data from files

In [2]:
jpath = 'data/house1.json' # path for json file
xpath = 'data/house2.xml' # path for xml file

## JSON DATA

In [3]:
j = pd.read_json(jpath) # use read json to parse json file
j.head(2)

,property_id,lat,lng,addr_street
0,29654,-37.783257,145.000198,2 Short Street
1,84314,-37.851406,145.003433,7 Pridham Street


## XML DATA

In [ ]:
x = pd.read_xml('xpath')

### Fix XML File

Syntax error : Start tag expected, '<' not found, line 1, column 1
so need to look in the file for xml format

In [4]:
with open(xpath,'r') as xfile:
    x = xfile.read()  # all xml
print(x[0:300],
x[-100:]) 

<property>
  <property_id>53857</property_id>
  <lat>-37.83034134</lat>
  <lng>144.9947815</lng>
  <addr_street>86 Balmain Street</addr_street>
</property>
<property>
  <property_id>68409</property_id>
  <lat>-37.856339</lat>
  <lng>145.02240700000004</lng>
  <addr_street>25 Clarendon Street</addr_s t>-37.741972</lat>
  <lng>145.018771</lng>
  <addr_street>126 Gower Street</addr_street>
</property>


Do not have wrapper 

In [5]:
x = "<property_list>" + x + "</property_list>"  # add wrapper

In [6]:
print(x[0:300],
x[-300:])

<property_list><property>
  <property_id>53857</property_id>
  <lat>-37.83034134</lat>
  <lng>144.9947815</lng>
  <addr_street>86 Balmain Street</addr_street>
</property>
<property>
  <property_id>68409</property_id>
  <lat>-37.856339</lat>
  <lng>145.02240700000004</lng>
  <addr_street>25 Clarendon 2265</property_id>
  <lat>-37.983546000000004</lat>
  <lng>145.055963</lng>
  <addr_street>39 Mundy Street</addr_street>
</property>
<property>
  <property_id>31804</property_id>
  <lat>-37.741972</lat>
  <lng>145.018771</lng>
  <addr_street>126 Gower Street</addr_street>
</property></property_list>


In [7]:
x = pd.read_xml(x)

XMLSyntaxError: xmlParseEntityRef: no name, line 3107, column 18 (<string>, line 3107)


Still have syntax error: no name, line 3107, column 18.
So check line 3107

In [8]:
# create list to access each line.
with open(xpath, "r") as xl: # open the file again
    alllines = xl.readlines() # use realines to make each line into element of list


In [9]:
# access line 3107 as index 3106
alllines[3106]

'  <addr_street>2&3/17-19 Browning Street</addr_street>\n'

***<** and **&** are strictly illegal in XML need to change to pre-defined entity*

There are 5 pre-defined entity references in XML:
|XML|Sign|Def|
|----|----|---------|
|\&lt;|<|less than|
|\&gt;|>|greater than|
|\&amp;|&|ampersand|
|\&apos;|'|apostrophe|
|\&quot;|"|quotation mark|

ref : https://www.w3schools.com/xml/xml_syntax.asp

In [10]:
x = x.replace("&","&amp;")

In [11]:
x = pd.read_xml(x)

In [12]:
x.head(2)

,property_id,lat,lng,addr_street
0,53857,-37.830341,144.994781,86 Balmain Street
1,68409,-37.856339,145.022407,25 Clarendon Street


## Combine JSON and XML df

In [13]:
print(x.shape,j.shape)

(1015, 4) (1019, 4)


In [14]:
property = pd.concat([x,j], ignore_index=True) # index need to be re-running

In [15]:
property.shape

(2034, 4)

# Cleaning the combined file

## check and manage duplicates

There is key column, property_id, which each property should have distince value. Therefore anchor in that column if have any duplicate, drop it out.

In [16]:
unique_property = property.drop_duplicates(subset='property_id').copy().reset_index(drop=True)

In [17]:
unique_property = property.drop_duplicates(subset='property_id').copy() #copy from original list

In [18]:
unique_property = unique_property.reset_index(drop=True) #reset index

In [19]:
unique_property.head(2)

,property_id,lat,lng,addr_street
0,53857,-37.830341,144.994781,86 Balmain Street
1,68409,-37.856339,145.022407,25 Clarendon Street


In [20]:
(unique_property== 'NA').sum() # check NA

property_id    0
lat            0
lng            0
addr_street    0
dtype: int64

# Find suburb of each property

## Clean suburb file

In [21]:
import geopandas as gpd

In [22]:
sub_path = "supplementary_data/Vic_suburb_boundary/VIC_LOCALITY_POLYGON_shp.shp"

In [23]:
sub = gpd.read_file(sub_path)
sub.head(2)

,LC_PLY_PID,DT_CREATE,DT_RETIRE,LOC_PID,VIC_LOCALI,VIC_LOCA_1,VIC_LOCA_2,VIC_LOCA_3,VIC_LOCA_4,VIC_LOCA_5,VIC_LOCA_6,VIC_LOCA_7,geometry
0,6670,2011-08-31,None,VIC2615,2012-04-27,None,UNDERBOOL,None,None,G,None,2,"POLYGON ((141.74552 -35.07229, 141.74552 -35.0..."
1,6671,2011-08-31,None,VIC1986,2012-04-27,None,NURRAN,None,None,G,None,2,"POLYGON ((148.66877 -37.39571, 148.66876 -37.3..."


In [24]:
subclean = sub[['VIC_LOCA_2','geometry']]
subclean.rename(columns={'VIC_LOCA_2':'suburb'}, inplace=True) # change column name

/var/folders/hn/mgkfr3qd4652k_9wn776wn4m0000gn/T/ipykernel_55790/993301998.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subclean.rename(columns={'VIC_LOCA_2':'suburb'}, inplace=True) # change column name


In [25]:
subclean.head(2)

,suburb,geometry
0,UNDERBOOL,"POLYGON ((141.74552 -35.07229, 141.74552 -35.0..."
1,NURRAN,"POLYGON ((148.66877 -37.39571, 148.66876 -37.3..."


## Find suburb of each property

Need to find suburb as the stations

In [26]:
from shapely.geometry import Point # use for plot in geometry

In [27]:
def findsuburbname(long,lat):
    '''
    use the location of property to find suburb
    '''
    
    proplocation = Point(long,lat)
    # locate where True(geo contain proplocation) then get suburb
    try:
        suburb = subclean.loc[subclean['geometry'].contains(proplocation), 'suburb'].values[0] 
    except:
        suburb = "NA"
    
    return suburb

In [28]:
unique_property['suburb'] = unique_property.apply(lambda x: findsuburbname(x['lng'],x['lat']), axis =1)

In [29]:
unique_property.head(2)

,property_id,lat,lng,addr_street,suburb
0,53857,-37.830341,144.994781,86 Balmain Street,CREMORNE
1,68409,-37.856339,145.022407,25 Clarendon Street,ARMADALE


In [30]:
unique_property.shape

(2012, 5)

# Fill LGA

## Extract LGA from pdf

In [31]:
lga_path = "supplementary_data/Lga_to_suburb.pdf"

In [32]:
from pdfminer.high_level import extract_text # for extract pdf text from pdfminer.six
lga = extract_text(lga_path)
lga[:200]

"BANYULE : ['Abbotsford', 'Airport West', 'Albert Park', 'Alphington'] \n\nBRIMBANK : ['Altona', 'Altona North', 'Armadale', 'Ascot Vale'] \n\nDAREBIN : ['Ashburton', 'Ashwood', 'Avondale Heights', 'Balacl"

Extract each LGA and its suburbs with RegEX

In [33]:
import re # for regex
lgasub_patt =r'[A-Z ]+ ?: ?\[[^\]]+\]'  # From first character to closing bracket

In [34]:
lgaandsub = re.findall(lgasub_patt, lga) 
lgaandsub[0:3]

["BANYULE : ['Abbotsford', 'Airport West', 'Albert Park', 'Alphington']",
 "BRIMBANK : ['Altona', 'Altona North', 'Armadale', 'Ascot Vale']",
 "DAREBIN : ['Ashburton', 'Ashwood', 'Avondale Heights', 'Balaclava']"]

The information is in string - convert to dict to work with easier

In [35]:
import ast # for convert to appropriate type

def stringtodict(lgasub):

    '''
    To convert string of LGA and subs to dict, identify key and  value 
    as the string is not in actual form of dict (no prenthesis) or list (has colon)
    '''
    k,v = lgasub.split(":",1) 
    key = k.strip()
    value = ast.literal_eval(v.strip())

    return {key:value}

In [36]:
lga_sub = {key:value for x in lgaandsub for key,value in stringtodict(x).items()}
lga_sub["BANYULE"]

['Abbotsford', 'Airport West', 'Albert Park', 'Alphington']

In [43]:
def findlga(suburb):
    """
    if suburb is match, return lga
    """
    for k,v in lga_sub.items(): # in each set of suburbs of LGA
        for i in v: # in each suburb in the LGA
            if suburb == i.upper(): # if suburb matched with suburb in the set
                 return k.upper() # lga is k
    return "NA"


In [44]:
findlga('ARMADALE')

'BRIMBANK'

In [45]:
unique_property['LGA'] = unique_property['suburb'].apply(findlga)

In [46]:
unique_property.head(2)

,property_id,lat,lng,addr_street,suburb,LGA
0,53857,-37.830341,144.994781,86 Balmain Street,CREMORNE,WELLINGTON
1,68409,-37.856339,145.022407,25 Clarendon Street,ARMADALE,BRIMBANK
